In [14]:
# -*- coding: utf-8 -*-
"""
Created on Mon Dec  9 23:29:24 2019

@author: bhaku
"""
from math import exp, sqrt
from random import gauss
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import scipy.stats
from cvxopt import matrix, solvers
from math import *
import random
from scipy.stats import norm

#from sklearn.linear_model import LinearRegression

In [5]:
class MonteCarlo:
    def __init__(self, S0, K, T, r, q, sigma, kappa=0, theta=0, xi=0, rho=0, V0=0,
                 underlying_process="geometric brownian motion"):
        self.underlying_process = underlying_process
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.q = q
        self.sigma = sigma
        self.kappa = kappa
        self.theta = theta
        self.rho = rho
        self.V0 = V0
        self.xi = xi

        self.value_results = None

    # view antithetic variates as a option of simulation method to reduce the variance    
    def simulate(self, n_trials, n_steps, antitheticVariates=False, boundaryScheme="geometric brownian motion"):

        dt = self.T / n_steps
        mu = self.r - self.q
        self.n_trials = n_trials
        self.n_steps = n_steps
        self.boundaryScheme = boundaryScheme

        if (self.underlying_process == "geometric brownian motion"):
            #             first_step_prices = np.ones((n_trials,1))*np.log(self.S0)
            log_price_matrix = np.zeros((n_trials, n_steps))
            normal_matrix = np.random.normal(size=(n_trials, n_steps))
            if (antitheticVariates == True):
                n_trials *= 2
                self.n_trials = n_trials
                normal_matrix = np.concatenate((normal_matrix, -normal_matrix), axis=0)
            cumsum_normal_matrix = normal_matrix.cumsum(axis=1)

            deviation_matrix = cumsum_normal_matrix * self.sigma * np.sqrt(dt) + \
                               (mu - self.sigma ** 2 / 2) * dt * np.arange(1, n_steps + 1)
            log_price_matrix = deviation_matrix + np.log(self.S0)
            price_matrix = np.exp(log_price_matrix)
            price_zero = (np.ones(n_trials) * self.S0)[:, np.newaxis]
            price_matrix = np.concatenate((price_zero, price_matrix), axis=1)
            self.price_matrix = price_matrix


        return price_matrix

    

    
    def BlackScholesPricer(self, option_type='c'):
        S = self.S0
        K = self.K
        T = self.T
        r = self.r
        q = self.q
        sigma = self.sigma
        d1 = (np.log(S / K) + (r - q) * T + 0.5 * sigma ** 2 * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        N = lambda x: sp.stats.norm.cdf(x)
        call = np.exp(-q * T) * S * N(d1) - np.exp(-r * T) * K * N(d2)
        put = call - np.exp(-q * T) * S + K * np.exp(-r * T)
        
        if (option_type == "c"):
            self.BSDelta = N(d1)
            self.BSPrice = call
            return call
        elif (option_type == "p"):
            self.BSDelta = -N(-d1)
            self.BSPrice = put
            return put
        else:
            print("please enter the option type: (c/p)")
        
        pass

    def MCPricer(self, option_type='c', isAmerican=False):
        price_matrix = self.price_matrix
        n_steps = self.n_steps
        n_trials = self.n_trials
        strike = self.K
        risk_free_rate = self.r
        time_to_maturity = self.T
        dt = time_to_maturity / n_steps
        if (option_type == "c"):
            payoff_fun = lambda x: np.maximum(x-strike,0)
        elif (option_type == "p"):
            payoff_fun = lambda x: np.maximum(strike-x, 0)
        else:
            print("please enter the option type: (c/p)")
            return

        if (isAmerican == False):

            payoff = payoff_fun(price_matrix[:, n_steps])
            #         vk = payoff*df
            value_results = payoff * np.exp(-risk_free_rate * time_to_maturity)
            self.payoff = payoff
        else:
            exercise_matrix = self.exercise_matrix
            t_exercise_array = dt * np.where(exercise_matrix == 1)[1]
            value_results = payoff_fun(price_matrix[np.where(exercise_matrix == 1)]) * np.exp(-risk_free_rate * t_exercise_array)
            
        regular_mc_price = np.average(value_results)
        self.mc_price = regular_mc_price
        self.value_results = value_results
        return regular_mc_price



    def standard_error(self):
        # can not apply to the OHMC since its result is not obtained by averaging
        # sample variance
        sample_var = np.var(self.value_results, ddof=1)
        std_estimate = np.sqrt(sample_var)
        standard_err = std_estimate / np.sqrt(self.n_trials)
        return standard_err

    def pricing(self, option_type='c', func_list=[lambda x: x ** 0, lambda x: x]):
        OHMC_price = self.OHMCPricer(option_type=option_type, func_list=func_list)
        regular_mc_price = self.MCPricer(option_type=option_type)
        black_sholes_price = self.BlackScholesPricer(option_type)
        return ({"OHMC": OHMC_price, "regular MC": regular_mc_price, "Black-Scholes": black_sholes_price})


def CBS(S, K, T, r, sigma,t, option_type):
    t=0
    t2t = T-t # time to maturity
    # Calculations for the solution to BSM equation
    dplus = (1 / (sigma * sqrt(t2t))) * ((log(S / K)) + (r + ((sigma ** 2) / 2)) * t2t)
    dminus = (1 / (sigma * sqrt(t2t))) * ((log(S / K)) + (r - (sigma ** 2) / 2) * t2t)

    # Calculating price of Call and Put
    if option_type == 'c':
        return S * norm.cdf(dplus) - K * exp(-r * t2t) * norm.cdf(dminus)
    elif option_type == 'p':
        return K * exp(-r * t2t) * norm.cdf(-dminus) - S * norm.cdf(-dplus)



def DBCV(S,r,sig,T,K,N,M,div,option_type,antithetic=False):
    dt = T/N
    nu = r - div - 0.5*(sig**2)
    nudt = nu*dt
    sigsdt = sig*np.sqrt(dt)
    erddt = np.exp((r-div)*dt)
    
    beta1 = -1
    
    sum_CT = 0
    sum_CT2 = 0
    if antithetic == False:
        for j in range(1,M): # For each simulation
        
            St = S
            cv = 0
        
            for i in range(1,N): # For each time step
                t = (i-1)*dt
                delta = CBS(St,K,T,r,sig,t,option_type)
                eps = np.random.normal(0, 1)
                Stn = St*np.exp(nudt+sigsdt*eps)
                cv1 = cv + delta*(Stn-St*erddt)
                St = Stn
        
        
            if option_type == 'c':
                CT = max(0, St - K) + beta1*cv1
                sum_CT = sum_CT + CT
                sum_CT2 = sum_CT2 + CT*CT
            elif option_type == 'p':
                CT = max(0, K - St) + beta1*cv1
                sum_CT = sum_CT + CT
                sum_CT2 = sum_CT2 + CT * CT
            else:
                break
    
    elif antithetic == True:
    
        for j in range(1,M): # For each simulation
        
            St1 = S
            St2=S
            cv1 = 0
            cv2=0
            for i in range(1,int(N)): # For each time step
                t = (i-1)*dt
                delta1 = CBS(St1,K,T,r,sig,t,option_type)
                delta2 = CBS(St2,K,T,r,sig,t,option_type)
                eps = np.random.normal(0, 1)
                Stn1 = St1*np.exp(nudt+sigsdt*eps)
                Stn2=St2*np.exp(nudt+sigsdt*eps)
                cv1 = cv1 + delta1*(Stn1-St1*erddt)
                cv2 = cv2 + delta2*(Stn2-St2*erddt)
                St1 = Stn1
                St2 = Stn2
        
            if option_type == 'c':
                CT = 0.5*(((max(0, St1 - K) + beta1*cv1)+(max(0, St2 - K) + beta1*cv2)))
                sum_CT = sum_CT + CT
                sum_CT2 = sum_CT2 + CT*CT
            elif option_type == 'p':
                CT = 0.5*(((max(0, K-St1 ) + beta1*cv1)+(max(0, K-St2 ) + beta1*cv2)))
                sum_CT = sum_CT + CT
                sum_CT2 = sum_CT2 + CT * CT
            else:
                break
    
     
    Value = sum_CT/M*np.exp(-r*T)
    SD = sqrt((sum_CT2 - sum_CT*sum_CT/M)*np.exp(-2*r*T)/(M-1))
    SE = SD/sqrt(M)
    list1=[Value,SD,SE]
    return list1   

In [21]:
S0 =100      # stock price at time zero
K=100      # exercise price
T =1       # years
r =0.05  
div=0.03     # risk-free rate
sigma = 0.2   # volatility
n_steps=300          # number of steps
sp.random.seed(1) # fix  random numbers
sig=0.2
M=10000
N=300
risk_free_rate = 0.05
dividend = 0.03
time_to_maturity = 1
volatility = 0.2
strike = 100
stock_price = 100
#option_tpe='c'

func_list = [lambda x: x**0, lambda x: x]

import timeit

def performance_comparison(n_trials,n_steps,option_type='c'):
    mc = MonteCarlo(stock_price,strike,time_to_maturity,risk_free_rate,dividend,volatility)
    # black scholes
    bs_start = timeit.default_timer()
    bs_price = mc.BlackScholesPricer(option_type)
    bs_sde_estimate = 0
    bs_end = timeit.default_timer()
    
    # regular MC
    rmc_start = timeit.default_timer()
    mc.simulate(n_trials,n_steps)
    rmc_price = mc.MCPricer(option_type)
    rmc_sde_estimate = mc.standard_error()
    rmc_end = timeit.default_timer()
    
    # regular MC with antithetic variates
    rmc_anti_start = timeit.default_timer()
    mc.simulate(int(n_trials/2),n_steps,antitheticVariates=True)
    rmc_anti_price = mc.MCPricer(option_type)
    rmc_anti_sde_estimate = mc.standard_error()
    rmc_anti_end = timeit.default_timer()
    
    # Delta-based MC
    dbmc_start = timeit.default_timer()
    list2=DBCV(S0,r,sig,T,K,N,M,div,option_type,antithetic=False)
    dbmc_price = list2[0]
    dbmc_sde_estimate = list2[2] 
    dbmc_end = timeit.default_timer()
    
    # Delta-based MC with antithetic variates
    dbmc_anti_start = timeit.default_timer()
    list3=DBCV(S0,r,sig,T,K,n_steps,n_trials,div,option_type,antithetic=True)
    dbmc_anti_price =list3[0]
    dbmc_anti_sde_estimate = list3[2]
    dbmc_anti_end = timeit.default_timer()
        
    bs_runtime = bs_end - bs_start
    rmc_runtime = rmc_end - rmc_start
    dbmc_runtime = dbmc_end - dbmc_start

    rmc_anti_runtime = rmc_anti_end - rmc_anti_start
    dbmc_anti_runtime = dbmc_anti_end - dbmc_anti_start

    
    rmc_err = np.abs(rmc_price - bs_price)
    dbmc_err = np.abs(dbmc_price - bs_price)

    rmc_anti_err = np.abs(rmc_anti_price - bs_price)
    dbmc_anti_err = np.abs(dbmc_anti_price - bs_price)

    
    
    result = {"method":["Black Scholes","MC","antithetic MC","DBMC","antithetic DBMC"],
              "option_type":[option_type]*5,
              "value":[bs_price,rmc_price,rmc_anti_price,dbmc_price,dbmc_anti_price],
              "runtime":[bs_runtime,rmc_runtime,rmc_anti_runtime,dbmc_runtime,dbmc_anti_runtime],
              "err":[0,rmc_err,rmc_anti_err,dbmc_err,dbmc_anti_err], 
              "sde estimate":[0,rmc_sde_estimate,rmc_anti_sde_estimate,dbmc_sde_estimate,
                              dbmc_anti_sde_estimate],
              "m":[n_trials]*5,
              "n":[n_steps]*5}
    return result

In [22]:
abc=performance_comparison(10000,300)
abcd=pd.DataFrame(abc)
abcde=performance_comparison(10000,300,'p')
abcdef=pd.DataFrame(abcde)

In [23]:
abcd

,method,option_type,value,runtime,err,sde estimate,m,n
0,Black Scholes,c,8.652529,0.000935,0.000000,0.000000,10000,300
1,MC,c,8.612278,0.458405,0.040251,0.132842,10000,300
2,antithetic MC,c,8.482440,0.402745,0.170088,0.131309,10000,300
3,DBMC,c,8.703328,1071.933685,0.050800,0.325763,10000,300
4,antithetic DBMC,c,9.090651,1069.017288,0.438123,3.697007,10000,300


In [24]:
abcdef

,method,option_type,value,runtime,err,sde estimate,m,n
0,Black Scholes,p,6.730918,0.000314,0.000000,0.000000,10000,300
1,MC,p,6.523748,0.202077,0.207170,0.092715,10000,300
2,antithetic MC,p,6.770450,0.173326,0.039532,0.094400,10000,300
3,DBMC,p,6.615986,467.272636,0.114931,0.135926,10000,300
4,antithetic DBMC,p,7.767573,936.278512,1.036656,1.502585,10000,300


Please note that antithetic Delta Based Monte Carlo takes a long time to converge and due to time issue i have to set the limit to 10000.If it was one million it would have given accurate results.